# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,las tablas,7.7667,-80.2833,86.18,99,93,5.73,PA,1722014514
1,1,doctor arroyo,23.6667,-100.1833,66.56,73,99,1.41,MX,1722014515
2,2,yellowknife,62.4560,-114.3525,74.41,46,40,13.80,CA,1722014483
3,3,coquimbo,-29.9533,-71.3436,58.73,77,0,13.80,CL,1722014515
4,4,puerto ayora,-0.7393,-90.3518,76.96,96,44,8.01,EC,1722014516
...,...,...,...,...,...,...,...,...,...,...
377,377,new martinsville,39.6445,-80.8576,80.92,52,77,1.99,US,1722014632
378,378,yallahs,17.8748,-76.5624,92.93,65,10,14.56,JM,1722014633
379,379,sonoita,31.8500,-112.8333,94.05,33,50,7.65,MX,1722014633
380,380,choix,26.7092,-108.3219,93.16,39,56,2.66,MX,1722014633


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
mapPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = 'OSM',
    frame_width = 896,
    frame_height = 504,
    
)


# Display the map
mapPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values #Between 69.8F and 80.6F, No clouds, wind speeds below or equal to 10.0662MPH
# IMPERIAL MEASUREMENTS RAHHH!!!!
FilteredTemps = city_data_df.loc[(city_data_df['Max Temp'] >= 69.8) & (city_data_df['Max Temp'] <= 80.6)]
FilteredWinds = FilteredTemps.loc[(city_data_df['Wind Speed'] <= 10.0662 )]
FilteredClouds = FilteredWinds.loc[(city_data_df['Cloudiness'] <=0)]

# Drop any rows with null values
IdealConditions = FilteredClouds.dropna(how='all')

# Display sample data
IdealConditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,jamestown,42.0970,-79.2353,75.52,44,0,8.05,US,1722014522
47,47,sturgeon bay,44.8342,-87.3770,73.76,53,0,8.05,US,1722014529
116,116,fort frances,48.5999,-93.4003,80.17,76,0,5.01,CA,1722014552
141,141,pacific grove,36.6177,-121.9166,75.36,79,0,5.01,US,1722014561
187,187,urbana,40.1106,-88.2073,78.89,60,0,4.00,US,1722014574
212,212,pocatello,42.8713,-112.4455,80.19,41,0,8.05,US,1722014582
222,222,international falls,48.6011,-93.4110,80.15,76,0,5.01,US,1722014585
238,238,rantoul,40.3084,-88.1559,78.82,63,0,0.00,US,1722014505
249,249,sheridan,44.8333,-106.9173,80.31,29,0,4.61,US,1722014593
277,277,jackson,42.4165,-122.8345,73.06,44,0,4.61,US,1722014602


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = IdealConditions.loc[:,['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,jamestown,US,42.0970,-79.2353,44,
47,sturgeon bay,US,44.8342,-87.3770,53,
116,fort frances,CA,48.5999,-93.4003,76,
141,pacific grove,US,36.6177,-121.9166,79,
187,urbana,US,40.1106,-88.2073,60,
212,pocatello,US,42.8713,-112.4455,41,
222,international falls,US,48.6011,-93.4110,76,
238,rantoul,US,40.3084,-88.1559,63,
249,sheridan,US,44.8333,-106.9173,29,
277,jackson,US,42.4165,-122.8345,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 8000
category = "accommodation.hotel"
limit = 10

params = {
    "categories": category,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # Not necessary
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    print(row)
# Display sample data
hotel_df

Starting hotel search
jamestown - nearest hotel: DoubleTree Jamestown
City          jamestown
Country              US
Lat              42.097
Lng            -79.2353
Humidity             44
Hotel Name             
Name: 22, dtype: object
sturgeon bay - nearest hotel: Stone Harbor
City          sturgeon bay
Country                 US
Lat                44.8342
Lng                -87.377
Humidity                53
Hotel Name                
Name: 47, dtype: object
fort frances - nearest hotel: The Sleepy Owl
City          fort frances
Country                 CA
Lat                48.5999
Lng               -93.4003
Humidity                76
Hotel Name                
Name: 116, dtype: object
pacific grove - nearest hotel: Pacific Grove Inn
City          pacific grove
Country                  US
Lat                 36.6177
Lng               -121.9166
Humidity                 79
Hotel Name                 
Name: 141, dtype: object
urbana - nearest hotel: Urbana Landmark Hotel
City         

,City,Country,Lat,Lng,Humidity,Hotel Name
22,jamestown,US,42.0970,-79.2353,44,DoubleTree Jamestown
47,sturgeon bay,US,44.8342,-87.3770,53,Stone Harbor
116,fort frances,CA,48.5999,-93.4003,76,The Sleepy Owl
141,pacific grove,US,36.6177,-121.9166,79,Pacific Grove Inn
187,urbana,US,40.1106,-88.2073,60,Urbana Landmark Hotel
212,pocatello,US,42.8713,-112.4455,41,Black Swan Inn
222,international falls,US,48.6011,-93.4110,76,Copper River Inn & Conference Centre
238,rantoul,US,40.3084,-88.1559,63,Holiday Inn Express
249,sheridan,US,44.8333,-106.9173,29,Comfort Inn & Suites
277,jackson,US,42.4165,-122.8345,44,Brookside


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
mapPlot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = 'OSM',
    frame_width = 896,
    frame_height = 504,
    hover_cols = ["Hotel Name"] 
)
# Display the map
mapPlot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)